In [ ]:
import os
import pandas as pd
import numpy as np

In [2]:
hares = pd.read_csv(os.path.join('data','knb-lter-bnz','55_Hare_Data_2012.txt'))
hares

,date,time,grid,trap,l_ear,r_ear,sex,age,weight,hindft,notes,b_key,session_id,study
0,11/26/1998,NaN,bonrip,1A,414D096A08,NaN,NaN,NaN,1370.0,160.0,NaN,917.0,51,Population
1,11/26/1998,NaN,bonrip,2C,414D320671,NaN,M,NaN,1430.0,NaN,NaN,936.0,51,Population
2,11/26/1998,NaN,bonrip,2D,414D103E3A,NaN,M,NaN,1430.0,NaN,NaN,921.0,51,Population
3,11/26/1998,NaN,bonrip,2E,414D262D43,NaN,NaN,NaN,1490.0,135.0,NaN,931.0,51,Population
4,11/26/1998,NaN,bonrip,3B,414D2B4B58,NaN,NaN,NaN,1710.0,150.0,NaN,933.0,51,Population
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3375,8/8/2002,18:00:00,bonrip,1b,1201,1202,NaN,NaN,1400.0,NaN,NaN,63.0,64,Population
3376,8/8/2002,6:00:00,bonrip,4b,1201,1202,NaN,NaN,NaN,NaN,NaN,63.0,64,Population
3377,8/7/2002,NaN,bonrip,4b,1217,1218,NaN,NaN,1000.0,134.0,NaN,69.0,64,Population
3378,8/8/2002,NaN,bonrip,6d,1217,1218,NaN,NaN,990.0,NaN,NaN,69.0,64,Population


In [3]:
hares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3380 entries, 0 to 3379
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        3380 non-null   object 
 1   time        264 non-null    object 
 2   grid        3380 non-null   object 
 3   trap        3368 non-null   object 
 4   l_ear       3332 non-null   object 
 5   r_ear       3211 non-null   object 
 6   sex         3028 non-null   object 
 7   age         1269 non-null   object 
 8   weight      2845 non-null   float64
 9   hindft      1633 non-null   float64
 10  notes       243 non-null    object 
 11  b_key       3333 non-null   float64
 12  session_id  3380 non-null   int64  
 13  study       3217 non-null   object 
dtypes: float64(3), int64(1), object(10)
memory usage: 369.8+ KB


In [3]:
hares.date = pd.to_datetime(hares.date)

In [6]:
hares.age.unique()

array([nan, 'J', 'A', 'a 1 yr.', 'a 3/4 yr.', 'a 1 yr', '1 yr', '1 yr.',
       '2 yrs.', '2 yrs', 'a 2 yrs.', '2.25 yrs', '3.5 yrs.', '3 yrs.',
       '2.5 yrs', '3.25 yrs.', 'A 1.5', '?', 'U', 'j', 'a', 'u', 'J 3/4',
       'A 3/4', 'A 1/2', '3/4/2013', '1/4/2013', '1/2/2013', '1', '1.25',
       '1.5'], dtype=object)

In [9]:
hares.loc[hares.age.str.contains('j', case=False, na=False), 'age'] = 'juvenile'
hares.loc[hares.age.str.contains('a', case=False, na=False), 'age'] = 'adult'

hares_ja = hares[hares.age.isin(['juvenile', 'adult'])].copy()
hares_ja.age.unique()

array(['juvenile', 'adult'], dtype=object)

In [44]:
np.arange(hares.date.dt.month.unique())

NameError: name 'np' is not defined

In [10]:
hares_ja['year'] = hares_ja.date.dt.year

In [23]:
hares_ja = hares_ja.groupby(['year', 'age']).size().reset_index(name='counts')

In [32]:
hares_ja

,year,age,counts
0,1999,adult,150
1,1999,juvenile,126
2,2000,adult,159
3,2000,juvenile,67
4,2001,adult,72
5,2001,juvenile,22
6,2002,adult,3
7,2003,adult,1
8,2003,juvenile,18
9,2004,juvenile,8


In [38]:
hares_ja = hares_ja.pivot(index='age', columns='year', values='counts')
hares_ja

year,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012
age,,,,,,,,,,,,,,
adult,150.0,159.0,72.0,3.0,1.0,NaN,95.0,101.0,13.0,84.0,10.0,21.0,45.0,5.0
juvenile,126.0,67.0,22.0,NaN,18.0,8.0,48.0,15.0,5.0,57.0,1.0,3.0,20.0,6.0


In [41]:
for year in range(1999,2013):
    juveniles = hares_ja.at['juvenile', year]
    adults = hares_ja.at['adult', year]
    print(f"{year}: juvenile proportion: {juveniles / (juveniles + adults):.2f}")

1999: juvenile proportion: 0.46
2000: juvenile proportion: 0.30
2001: juvenile proportion: 0.23
2002: juvenile proportion: nan
2003: juvenile proportion: 0.95
2004: juvenile proportion: nan
2005: juvenile proportion: 0.34
2006: juvenile proportion: 0.13
2007: juvenile proportion: 0.28
2008: juvenile proportion: 0.40
2009: juvenile proportion: 0.09
2010: juvenile proportion: 0.12
2011: juvenile proportion: 0.31
2012: juvenile proportion: 0.55


In [18]:
hares.b_key.value_counts()

b_key
285.0    42
92.0     42
961.0    36
729.0    28
233.0    24
         ..
419.0     1
418.0     1
340.0     1
342.0     1
68.0      1
Name: count, Length: 1026, dtype: int64

In [19]:
hares[hares.b_key > 1]

,date,time,grid,trap,l_ear,r_ear,sex,age,weight,hindft,notes,b_key,session_id,study
0,1998-11-26,NaN,bonrip,1A,414D096A08,NaN,NaN,NaN,1370.0,160.0,NaN,917.0,51,Population
1,1998-11-26,NaN,bonrip,2C,414D320671,NaN,M,NaN,1430.0,NaN,NaN,936.0,51,Population
2,1998-11-26,NaN,bonrip,2D,414D103E3A,NaN,M,NaN,1430.0,NaN,NaN,921.0,51,Population
3,1998-11-26,NaN,bonrip,2E,414D262D43,NaN,NaN,NaN,1490.0,135.0,NaN,931.0,51,Population
4,1998-11-26,NaN,bonrip,3B,414D2B4B58,NaN,NaN,NaN,1710.0,150.0,NaN,933.0,51,Population
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3375,2002-08-08,18:00:00,bonrip,1b,1201,1202,NaN,NaN,1400.0,NaN,NaN,63.0,64,Population
3376,2002-08-08,6:00:00,bonrip,4b,1201,1202,NaN,NaN,NaN,NaN,NaN,63.0,64,Population
3377,2002-08-07,NaN,bonrip,4b,1217,1218,NaN,NaN,1000.0,134.0,NaN,69.0,64,Population
3378,2002-08-08,NaN,bonrip,6d,1217,1218,NaN,NaN,990.0,NaN,NaN,69.0,64,Population


In [20]:
hares[hares.b_key == 285]

,date,time,grid,trap,l_ear,r_ear,sex,age,weight,hindft,notes,b_key,session_id,study
1202,2007-08-23,NaN,bonrip,2C,2085,2086,M,NaN,1280.0,118.0,NaN,285.0,82,Population
1224,2007-08-24,NaN,bonrip,2C,2085,2086,M,NaN,1180.0,NaN,NaN,285.0,82,Population
1245,2007-11-13,NaN,bonrip,1c,2085,2086,M,NaN,1480.0,136.0,NaN,285.0,83,Population
1257,2007-11-14,NaN,bonrip,1b,2085,2086,M,NaN,1600.0,NaN,NaN,285.0,83,Population
1271,2007-11-15,NaN,bonrip,2d,2085,2086,M,NaN,1560.0,NaN,NaN,285.0,83,Population
1283,2007-11-16,NaN,bonrip,1d,2085,2086,M,NaN,1510.0,NaN,NaN,285.0,83,Population
1304,2008-03-18,NaN,bonrip,3e,2085,2086,M,NaN,1610.0,143.0,NaN,285.0,84,Population
1316,2008-03-19,NaN,bonrip,2d,2085,2086,M,NaN,1540.0,NaN,NaN,285.0,84,Population
1329,2008-03-20,NaN,bonrip,1b,2085,2086,M,NaN,1580.0,NaN,NaN,285.0,84,Population
1348,2008-03-21,NaN,bonrip,3e,2085,2086,M,NaN,1550.0,NaN,NaN,285.0,84,Population
